To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Follow on LinkedIn + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

this code was developed using Unsloth <div> <a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
</div>

### Installation

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.247600
2,2.323100
3,2.204200
4,2.311800
5,2.209900
6,2.532100
7,1.959500
8,2.445300
9,2.077100
10,2.547900


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

462.7198 seconds used for training.
7.71 minutes used for training.
Peak reserved memory = 7.922 GB.
Peak reserved memory for training = 1.938 GB.
Peak reserved memory % of max memory = 53.716 %.
Peak reserved memory for training % of max memory = 13.141 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [6]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34\n\n### Instruction:\nThe following list of numbers has been sorted into ascending order. The next number in the sequence is missing. Insert the missing number.\n\n### Input:\n1, 3, 4, 5, 6, 8, 10, 12\n\n###']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [10]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        """You are given an invoice, extract the data and return them in the following order: invoice object with client name, client address, seller name, seller name, invoice number, invoice date, due date. Item purchased objects with their description, quantity , and total price. quantity can be also QTY. subtotal object with tax(if applicable), discount(if applicable), and total. Payment instructions with due date, bank name, account number, and payment method. in JSON format for direct use without any extra text. additionally, don't write json before the text also don't add comments """, # instruction
        """ Invoice no: 74898182

Date of issue:

Seller:

Myers-Holmes
Unit 0245 Box 5002
DPO AA 31147

Tax Id: 977-73-2842
IBAN: GB14MWCV18105652308620

ITEMS

No. Description

1 12" White Marble Side Coffee
Table Top Jasper Elephant Inlay
Garden Decor W026

2. 16 Inches Marble Corner Table
Top Inlay Coffee Table with
Multi Color Gemstones

3: 4'x2' Marble Dining Table Top

Pietra Dura Birds Inlay Art
Furniture Decors B444

SUMMARY

Total

Qty

4,00

3,00

1,00

VAT [%]

10%

10/24/2017

UM

each

each

each

Client:

Net price

247,78

238,00

1 840,10

Net worth
3 545,22

$ 3 545,22

Jennings, Hall and Mccoy
096 Ryan Knolls Apt. 299
Port Robertburgh, PA 98040

Tax Id: 924-88-9771

Net worth VAT [%] Gross
worth

991,12 10% 1 090,23
714,00 10% 785,40

1 840,10 10% 2 024,11
VAT Gross worth

354,52 3 899,74

$ 354,52 $3 899,74
_x000c_""", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 564)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are given an invoice, extract the data and return them in the following order: invoice object with client name, client address, seller name, seller name, invoice number, invoice date, due date. Item purchased objects with their description, quantity, and total price. quantity can be also QTY. subtotal object with tax(if applicable), discount(if applicable), and total. Payment instructions with due date, bank name, account number, and payment method. in JSON format for direct use without any extra text. additionally, don't write json before the text also don't add comments 

### Input:
 Invoice no: 74898182

Date of issue:

Seller:

Myers-Holmes
Unit 0245 Box 5002
DPO AA 31147

Tax Id: 977-73-2842
IBAN: GB14MWCV18105652308620

ITEMS

No. Description

1 12" White Marble Side Coffee
Table Top Ja

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [12]:
model.save_pretrained("Meta-Llama-3.1-8B")  # Local saving
tokenizer.save_pretrained("Meta-Llama-3.1-8B")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('Meta-Llama-3.1-8B/tokenizer_config.json',
 'Meta-Llama-3.1-8B/special_tokens_map.json',
 'Meta-Llama-3.1-8B/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [15]:
assert model.config._name_or_path == tokenizer.name_or_path, "Model and tokenizer must match."


In [25]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="Meta-Llama-3.1-8B",  # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

# Debugging enhancements start here

# Validate tokenizer and model compatibility
assert model.config._name_or_path == tokenizer.name_or_path, "Model and tokenizer mismatch!"

# Check the input before sending to the model
print("Generating input for model...")
formatted_prompt = alpaca_prompt.format(
    """You are given an invoice, extract the data and return them in the following order: invoice object with client name, client address, seller name, seller name, invoice number, invoice date, due date. Item purchased objects with their description, quantity , and total price. quantity can be also QTY. subtotal object with tax(if applicable), discount(if applicable), and total. Payment instructions with due date, bank name, account number, and payment method. in JSON format for direct use without any extra text. additionally, don't write json before the text also don't add comments """,  # instruction
    """ Invoice no: 74898182

Date of issue:

Seller:

Myers-Holmes
Unit 0245 Box 5002
DPO AA 31147

Tax Id: 977-73-2842
IBAN: GB14MWCV18105652308620

ITEMS

No. Description

1 12" White Marble Side Coffee
Table Top Jasper Elephant Inlay
Garden Decor W026

2. 16 Inches Marble Corner Table
Top Inlay Coffee Table with
Multi Color Gemstones

3: 4'x2' Marble Dining Table Top

Pietra Dura Birds Inlay Art
Furniture Decors B444

SUMMARY

Total

Qty

4,00

3,00

1,00

VAT [%]

10%

10/24/2017

UM

each

each

each

Client:

Net price

247,78

238,00

1 840,10

Net worth
3 545,22

$ 3 545,22

Jennings, Hall and Mccoy
096 Ryan Knolls Apt. 299
Port Robertburgh, PA 98040

Tax Id: 924-88-9771

Net worth VAT [%] Gross
worth

991,12 10% 1 090,23
714,00 10% 785,40

1 840,10 10% 2 024,11
VAT Gross worth

354,52 3 899,74

$ 354,52 $3 899,74
_x000c_ """,  # input
    "",  # output - leave this blank for generation!
)

# Check special tokens
print("Special tokens:", tokenizer.special_tokens_map)
print("Special token IDs:", tokenizer.all_special_ids)

# Reinitialize tokenizer if necessary
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("your-model-checkpoint")

# Check and clip token IDs
max_vocab_id = tokenizer.vocab_size - 1
if inputs['input_ids'].max().item() >= tokenizer.vocab_size:
    print("Clipping out-of-range token IDs...")
    inputs['input_ids'] = inputs['input_ids'].clamp(0, max_vocab_id)

# Decode tokenized input to debug
decoded_text = tokenizer.decode(inputs['input_ids'][0])
print("Decoded tokenized input:", decoded_text)



Generating input for model...
Special tokens: {'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|finetune_right_pad_id|>'}
Special token IDs: [128000, 128001, 128004]
Clipping out-of-range token IDs...
Decoded tokenized input: 锦Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are given an invoice, extract the data and return them in the following order: invoice object with client name, client address, seller name, seller name, invoice number, invoice date, due date. Item purchased objects with their description, quantity, and total price. quantity can be also QTY. subtotal object with tax(if applicable), discount(if applicable), and total. Payment instructions with due date, bank name, account number, and payment method. in JSON format for direct use without any extra text. additionally, don't write json before the text also 

In [32]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "kk", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Path to your uploaded model
model_path = "Meta-Llama-3.1-8B"  # Replace with the actual path to your model directory

# Load model and tokenizer from the local path
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Add special tokens if missing
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "<pad>"})
    model.resize_token_embeddings(len(tokenizer))

# Debugging: Validate model and tokenizer
print("Model and tokenizer loaded successfully.")
print("Special tokens:", tokenizer.special_tokens_map)
print("Special token IDs:", tokenizer.all_special_ids)

# Generate inputs from the formatted prompt
inputs = tokenizer(
    [
        alpaca_prompt.format(
            """You are given an invoice, extract the data and return them in the following order: invoice object with client name, client address, seller name, seller name, invoice number, invoice date, due date. Item purchased objects with their description, quantity , and total price. quantity can be also QTY. subtotal object with tax(if applicable), discount(if applicable), and total. Payment instructions with due date, bank name, account number, and payment method. in JSON format for direct use without any extra text. additionally, don't write json before the text also don't add comments """,  # instruction
            """ Invoice no: 74898182 ...

            ... (invoice content here) ...
            """,  # Input text
        )
    ],
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=model.config.max_position_embeddings
)

# Debugging: Validate token IDs
print("Max token ID:", inputs["input_ids"].max().item())
print("Min token ID:", inputs["input_ids"].min().item())
print(f"Input tensor shape: {inputs['input_ids'].shape}")

# Move model and inputs to GPU for generation
try:
    print("Moving model and inputs to GPU...")
    inputs = inputs.to("cuda")
    model = model.to("cuda")
except RuntimeError as e:
    print(f"Error moving to GPU: {e}")
    raise

# Generate output
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

try:
    print("Generating output...")
    output = model.generate(**inputs, streamer=text_streamer, max_new_tokens=564)
    print("Generated output:")
    print(tokenizer.decode(output[0], skip_special_tokens=True))
except RuntimeError as e:
    print(f"Error during generation: {e}")
    raise


`low_cpu_mem_usage` was None, now default to True since model is quantized.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in Cloude or content files here in Colab.

And we're done! If you have any questions , we have a [LinkedIn ](https://discord.gg/unsloth) Account! If you find any bugs or want to keep updated with the latest stuff, or need help, join projects etc, feel free to contact us!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Don't hesitate to contact us if you need help + ⭐️ <i>Star us on <a href="https://github.com/MAD-SAM22/Fine-Tuned-LLMs-For-OCR-/tree/main">Github</a> </i> ⭐️
</div>


## saving Data to Drive

In [36]:
# Function to save models and files to cloud storage
def save_to_cloud():
    import shutil  # Ensure shutil is imported
    from google.colab import drive
    import os  # Import os for path operations

    drive.mount('/content/drive')

    # Define the directory to save
    cloud_dir = '/content/drive/My Drive/Meta-Llama-3.1-8B-4000 record'

    # Create the directory if it doesn't exist
    shutil.os.makedirs(cloud_dir, exist_ok=True)

    # Save all relevant files for DistilBERT
    for filename in [
        # '/content/results_distilbert.csv',
        # '/content/distilbert-base-uncased_size_1000',
        # '/content/distilbert-base-uncased_size_2000',
        # '/content/distilbert-base-uncased_size_1250',
        # '/content/distilbert-base-uncased_size_1500',
        # '/content/distilbert-base-uncased_size_250',
        # '/content/distilbert-base-uncased_size_750',
        # '/content/distilbert-base-uncased_size_500',
        # '/content/wandb',
        '/content/Meta-Llama-3.1-8B']:
        destination = shutil.os.path.join(cloud_dir, shutil.os.path.basename(filename))
        if shutil.os.path.exists(filename):
            if shutil.os.path.isdir(filename):
                # Check if the destination already exists
                if not shutil.os.path.exists(destination):
                    shutil.copytree(filename, destination)
                else:
                    print(f"Directory {destination} already exists, skipping.")
            else:
                # Copy individual file
                shutil.copy(filename, destination)
                print(f"File {filename} copied successfully to {destination}")
        else:
            print(f"File or directory {filename} does not exist, skipping.")

    print(f"All models and results have been saved to {cloud_dir}")


# Save everything to cloud storage
save_to_cloud()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
All models and results have been saved to /content/drive/My Drive/Meta-Llama-3.1-8B-4000 record
